In [1]:
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'day 05 intcode v2'))
	print(os.getcwd())
except:
	pass

In [2]:
import sys
from collections import namedtuple
Output = namedtuple('Output',['code', 'noun', 'verb','error'])

f=open('input.txt').read()
memory = tuple(int(i) for i in f.split(',')) # let's make it immutable as a tuple
class Computer:   
    def __init__(self,code, noun,verb, inputvalue):
        self.code, self.noun, self.verb, self.input, self.p = code, noun, verb, inputvalue, 0
        self.opcode = {
            1:(self.summing,3),
            2:(self.multiply,3),
            3:(self.saveinput,1),
            4:(self.outputparam,1),
            5:(self.jump_if_true,2),
            6:(self.jump_if_false,2),
            7:(self.less,3),
            8:(self.eq,3),
            99:(self.exit,0)} 

    def restoregap(self):
        self.code[1] = self.noun
        self.code[2] = self.verb

    def summing(self,i,p): 
        n = self.convert_to_param(i[:-1],p)
        self.code[self.imm(i[-1])] = n[0]+n[1] 
    def multiply(self,i,p):
        n = self.convert_to_param(i[:-1],p)
        self.code[self.imm(i[-1])] = n[0]*n[1]
    def saveinput(self,i,p):  
        self.code[self.imm(i[-1])] = self.input
    def outputparam(self,i,p): 
        n = self.convert_to_param(i,p)
        print('opcode 4 ',n)
    def jump_if_true(self,i,p):  
        n = self.convert_to_param(i,p)
        if n[0] != 0: self.p = n[1]-3
    def jump_if_false(self,i,p):  
        n = self.convert_to_param(i,p)
        if n[0] == 0: self.p = n[1]-3
    def less(self,i,p): 
        n = self.convert_to_param(i[:-1],p)
        if n[0]<n[1]: self.code[self.imm(i[-1])] = 1
        else: self.code[self.imm(i[-1])] = 0
    def eq(self,i,p): 
        n = self.convert_to_param(i[:-1],p)
        if n[0]==n[1]: self.code[self.imm(i[-1])] = 1
        else: self.code[self.imm(i[-1])] = 0
    def exit(self,i,p): print('should not be called')

    def analyze_opcode(self, val):
        op = val[-2:]
        parameters = val[-3::-1]
        return(op,parameters)
    def imm(self,x):return int(self.code[x])
    def par(self,x):return int(self.code[self.code[x]])
    def convert_to_param(self,ind,p):
        res = []
        for i in range(len(ind)):    
            if i<len(p) and p[i]=='1': res.append(self.imm(ind[i]))
            else: res.append(self.par(ind[i]))
        res= [int(r) for r in res]
        return res

    def run(self):
        # self.restoregap() skip this step for day 5
        while True:
            try:
                op,parameters = self.analyze_opcode(str(self.code[self.p]))
                instruction, num_param = self.opcode[int(op)]
                if instruction == self.exit:
                    out = Output(self.code,self.noun,self.verb,None)
                    return out
                indices = list(range(self.p+1, self.p+1+num_param))
                print('ins',self.code[self.p],indices, num_param,instruction)
                instruction(indices,parameters)
                self.p += num_param+1
            except Exception as e: 
                out = Output(self.code, self.noun,self.verb,e)
                return out

In [32]:
inputvalue = 5
noun, verb = 0,0

c = Computer([5,5,2,3,0,100,0,0,0,0],noun,verb,5)
i = [0,1,2]
p = '111'
assert c.convert_to_param(i[:-1],p) == [5,5]
p = '000'
assert c.convert_to_param(i[:-1],p) == [100,100]
c.summing([0,1,2],'000')
assert c.code[2]==200
c.summing([0,1,3],'111')
assert c.code[3]==10
print(c.code[0:4])

[5, 5, 200, 10]


In [33]:
c = Computer(list(memory),noun,verb,5)
c.run()

ins 3 [1] 1 <bound method Computer.saveinput of <__main__.Computer object at 0x000001D84E6D3C18>>
ins 1 [3, 4, 5] 3 <bound method Computer.summing of <__main__.Computer object at 0x000001D84E6D3C18>>
ins 1105 [7, 8] 2 <bound method Computer.jump_if_true of <__main__.Computer object at 0x000001D84E6D3C18>>
ins 1105 [239, 240] 2 <bound method Computer.jump_if_true of <__main__.Computer object at 0x000001D84E6D3C18>>
ins 1105 [242, 243] 2 <bound method Computer.jump_if_true of <__main__.Computer object at 0x000001D84E6D3C18>>
ins 1005 [248, 249] 2 <bound method Computer.jump_if_true of <__main__.Computer object at 0x000001D84E6D3C18>>
ins 1005 [251, 252] 2 <bound method Computer.jump_if_true of <__main__.Computer object at 0x000001D84E6D3C18>>
ins 1106 [257, 258] 2 <bound method Computer.jump_if_false of <__main__.Computer object at 0x000001D84E6D3C18>>
ins 1106 [260, 261] 2 <bound method Computer.jump_if_false of <__main__.Computer object at 0x000001D84E6D3C18>>
ins 1006 [266, 267] 2 <bo

Output(code=[314, 225, 1, 225, 6, 6, 1105, 1, 238, 225, 104, 0, 101, 14, 135, 224, 101, -69, 224, 224, 4, 224, 1002, 223, 8, 223, 101, 3, 224, 224, 1, 224, 223, 223, 102, 90, 169, 224, 1001, 224, -4590, 224, 4, 224, 1002, 223, 8, 223, 1001, 224, 1, 224, 1, 224, 223, 223, 1102, 90, 45, 224, 1001, 224, -4050, 224, 4, 224, 102, 8, 223, 223, 101, 5, 224, 224, 1, 224, 223, 223, 1001, 144, 32, 224, 101, -72, 224, 224, 4, 224, 102, 8, 223, 223, 101, 3, 224, 224, 1, 223, 224, 223, 1102, 36, 93, 225, 1101, 88, 52, 225, 1002, 102, 38, 224, 101, -3534, 224, 224, 4, 224, 102, 8, 223, 223, 101, 4, 224, 224, 1, 223, 224, 223, 1102, 15, 57, 225, 1102, 55, 49, 225, 1102, 11, 33, 225, 1101, 56, 40, 225, 1, 131, 105, 224, 101, -103, 224, 224, 4, 224, 102, 8, 223, 223, 1001, 224, 2, 224, 1, 224, 223, 223, 1102, 51, 39, 225, 1101, 45, 90, 225, 2, 173, 139, 224, 101, -495, 224, 224, 4, 224, 1002, 223, 8, 223, 1001, 224, 5, 224, 1, 223, 224, 223, 1101, 68, 86, 224, 1001, 224, -154, 224, 4, 224, 102, 8, 223,